<a href="https://colab.research.google.com/github/abinayap215-web/hypothesis/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate huggingface_hub

from huggingface_hub import login
login()

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

conversation = []

print("Chatbot initialized. You can start chatting (type 'exit' to stop).")

while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        print("Exiting chat.")
        break

    # Add user's message to the conversation history
    conversation.append({"role": "user", "content": user_input})

    # Format the conversation for the LLaMA-2 chat model using its chat template
    inputs = tokenizer.apply_chat_template(
        conversation,
        return_tensors="pt",
        add_special_tokens=False,
        add_generation_prompt=True
    )

    # Move input tensors to the model's device (GPU/CPU)
    inputs = inputs.to(model.device) if hasattr(inputs, "to") else {
        k: v.to(model.device) for k, v in inputs.items()
    }

    # Generate a response from the model
    outputs = model.generate(
        **inputs,
        max_new_tokens=150
    )

    # Decode only the newly generated tokens (skip the prompt tokens)
    input_len = inputs["input_ids"].shape[-1]
    response = tokenizer.decode(
        outputs[0][input_len:],
        skip_special_tokens=True
    )

    print(f"Chatbot: {response}")

    # Add assistant's response to the conversation history
    conversation.append({"role": "assistant", "content": response})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Chatbot initialized. You can start chatting (type 'exit' to stop).
